### Feature Engineering

In [72]:
import pandas as pd

In [73]:
# Read the data from the CSV file
df_calls = pd.read_csv("../data/clean/calls.csv")
df_messages = pd.read_csv("../data/clean/messages.csv")
df_internet = pd.read_csv("../data/clean/internet.csv")
df_users = pd.read_csv("../data/clean/users.csv")
df_plans = pd.read_csv("../data/clean/plans.csv")

In [74]:
# Pasar Dates a datetime
df_calls["call_date"] = pd.to_datetime(df_calls["call_date"])
df_internet["session_date"] = pd.to_datetime(df_internet["session_date"])
df_messages["message_date"] = pd.to_datetime(df_messages["message_date"])
df_users["reg_date"] = pd.to_datetime(df_users["reg_date"])

In [75]:
# 1. Unir "Users" con "Plans" usando la columna "plan" que está en "Users" y en "Plans"
df_merged = pd.merge(df_users[['user_id', 'age', 'plan']], df_plans, left_on='plan', right_on='plan_name', how='left')


In [76]:
df_merged = df_merged.drop(columns=['plan_name'])
df_merged = df_merged.drop(columns=['plan'])


In [77]:
df_merged

,user_id,age,messages_included,mb_per_month_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute
0,1000,45,1000,30720,3000,70,7,0.01,0.01
1,1001,28,50,15360,500,20,10,0.03,0.03
2,1002,36,50,15360,500,20,10,0.03,0.03
3,1003,52,50,15360,500,20,10,0.03,0.03
4,1004,40,50,15360,500,20,10,0.03,0.03
...,...,...,...,...,...,...,...,...,...
495,1495,67,50,15360,500,20,10,0.03,0.03
496,1496,49,50,15360,500,20,10,0.03,0.03
497,1497,49,1000,30720,3000,70,7,0.01,0.01
498,1498,51,50,15360,500,20,10,0.03,0.03


In [78]:
df_called_merge = df_calls.drop(columns=['id'])

In [79]:
# Saca el mes de la fecha
df_called_merge['month'] = df_called_merge['call_date'].dt.month
df_called_merge = df_called_merge.drop(columns=['call_date'])

In [80]:
# Suma la duración de las llamadas por usuario y mes
df_called_merge = df_called_merge.groupby(['user_id', 'month'])['duration'].sum().reset_index()

In [81]:
# 2. Unir "Calls" con "df_merged" usando la columna "user_id" que está en "Calls" y en "df_merged"
df_called_merge = pd.merge(df_called_merge, df_merged, on='user_id', how='left')

In [82]:
df_called_merge

,user_id,month,duration,age,messages_included,mb_per_month_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute
0,1000,12,116.83,45,1000,30720,3000,70,7,0.01,0.01
1,1001,8,171.14,28,50,15360,500,20,10,0.03,0.03
2,1001,9,297.69,28,50,15360,500,20,10,0.03,0.03
3,1001,10,374.11,28,50,15360,500,20,10,0.03,0.03
4,1001,11,374.02,28,50,15360,500,20,10,0.03,0.03
...,...,...,...,...,...,...,...,...,...,...,...
2251,1498,12,324.77,51,50,15360,500,20,10,0.03,0.03
2252,1499,9,306.95,37,50,15360,500,20,10,0.03,0.03
2253,1499,10,338.25,37,50,15360,500,20,10,0.03,0.03
2254,1499,11,287.91,37,50,15360,500,20,10,0.03,0.03


In [83]:
# Calcular costos adicionales por sobreuso
df_called_merge['costo_extra_llamadas'] = (
    (df_called_merge['duration'] - df_called_merge['minutes_included'])
    .clip(lower=0) * df_called_merge['usd_per_minute']
)

In [84]:
df_called_merge

,user_id,month,duration,age,messages_included,mb_per_month_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute,costo_extra_llamadas
0,1000,12,116.83,45,1000,30720,3000,70,7,0.01,0.01,0.0
1,1001,8,171.14,28,50,15360,500,20,10,0.03,0.03,0.0
2,1001,9,297.69,28,50,15360,500,20,10,0.03,0.03,0.0
3,1001,10,374.11,28,50,15360,500,20,10,0.03,0.03,0.0
4,1001,11,374.02,28,50,15360,500,20,10,0.03,0.03,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2251,1498,12,324.77,51,50,15360,500,20,10,0.03,0.03,0.0
2252,1499,9,306.95,37,50,15360,500,20,10,0.03,0.03,0.0
2253,1499,10,338.25,37,50,15360,500,20,10,0.03,0.03,0.0
2254,1499,11,287.91,37,50,15360,500,20,10,0.03,0.03,0.0


In [85]:
df_calls_final = df_called_merge[['user_id', 'month', 'duration', 'costo_extra_llamadas']]


In [86]:
df_calls_final

,user_id,month,duration,costo_extra_llamadas
0,1000,12,116.83,0.0
1,1001,8,171.14,0.0
2,1001,9,297.69,0.0
3,1001,10,374.11,0.0
4,1001,11,374.02,0.0
...,...,...,...,...
2251,1498,12,324.77,0.0
2252,1499,9,306.95,0.0
2253,1499,10,338.25,0.0
2254,1499,11,287.91,0.0


In [87]:
df_internet_merge = df_internet.drop(columns=['id'])

In [88]:
df_internet_merge

,user_id,session_date,mb_used
0,1000,2018-12-29,89.86
1,1000,2018-12-31,0.00
2,1000,2018-12-28,660.40
3,1000,2018-12-26,270.99
4,1000,2018-12-27,880.22
...,...,...,...
104396,1499,2018-10-20,218.06
104397,1499,2018-12-30,304.72
104398,1499,2018-09-22,292.75
104399,1499,2018-12-07,0.00


In [89]:
# Saca el mes de la fecha
df_internet_merge['month'] = df_internet_merge['session_date'].dt.month
df_internet_merge = df_internet_merge.drop(columns=['session_date'])

In [90]:
df_internet_merge

,user_id,mb_used,month
0,1000,89.86,12
1,1000,0.00,12
2,1000,660.40,12
3,1000,270.99,12
4,1000,880.22,12
...,...,...,...
104396,1499,218.06,10
104397,1499,304.72,12
104398,1499,292.75,9
104399,1499,0.00,12


In [91]:
df_internet_merge = df_internet_merge.groupby(['user_id', 'month'])['mb_used'].sum().reset_index()

In [92]:
df_internet_merge

,user_id,month,mb_used
0,1000,12,1901.47
1,1001,8,6919.15
2,1001,9,13314.82
3,1001,10,22330.49
4,1001,11,18504.30
...,...,...,...
2272,1498,12,23137.69
2273,1499,9,12984.76
2274,1499,10,19492.43
2275,1499,11,15580.97


In [93]:
df_internet_merge = pd.merge(df_internet_merge, df_merged, on='user_id', how='left')

In [94]:
df_internet_merge['costo_extra_datos'] = (
    ((df_internet_merge['mb_used'] - df_internet_merge['mb_per_month_included']) / 1024)
    .clip(lower=0) * df_internet_merge['usd_per_gb']
)

In [95]:
df_internet_merge

,user_id,month,mb_used,age,messages_included,mb_per_month_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute,costo_extra_datos
0,1000,12,1901.47,45,1000,30720,3000,70,7,0.01,0.01,0.000000
1,1001,8,6919.15,28,50,15360,500,20,10,0.03,0.03,0.000000
2,1001,9,13314.82,28,50,15360,500,20,10,0.03,0.03,0.000000
3,1001,10,22330.49,28,50,15360,500,20,10,0.03,0.03,68.071191
4,1001,11,18504.30,28,50,15360,500,20,10,0.03,0.03,30.706055
...,...,...,...,...,...,...,...,...,...,...,...,...
2272,1498,12,23137.69,51,50,15360,500,20,10,0.03,0.03,75.954004
2273,1499,9,12984.76,37,50,15360,500,20,10,0.03,0.03,0.000000
2274,1499,10,19492.43,37,50,15360,500,20,10,0.03,0.03,40.355762
2275,1499,11,15580.97,37,50,15360,500,20,10,0.03,0.03,2.157910


In [96]:
df_internet_final = df_internet_merge[['user_id', 'month', 'mb_used', 'costo_extra_datos']]

In [97]:
df_messages_merge = df_messages.drop(columns=['id'])

In [98]:
# Saca el mes de la fecha
df_messages_merge['month'] = df_messages_merge['message_date'].dt.month
df_messages_merge = df_messages_merge.drop(columns=['message_date'])

In [99]:
# Agrupa
df_messages_merge = df_messages_merge.groupby(['user_id', 'month']).size().reset_index(name='sms_count')

In [100]:
df_messages_merge

,user_id,month,sms_count
0,1000,12,11
1,1001,8,30
2,1001,9,44
3,1001,10,53
4,1001,11,36
...,...,...,...
1754,1496,9,21
1755,1496,10,18
1756,1496,11,13
1757,1496,12,11


In [101]:
# merge
df_messages_merge = pd.merge(df_messages_merge, df_merged, on='user_id', how='left')

In [102]:
df_messages_merge

,user_id,month,sms_count,age,messages_included,mb_per_month_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute
0,1000,12,11,45,1000,30720,3000,70,7,0.01,0.01
1,1001,8,30,28,50,15360,500,20,10,0.03,0.03
2,1001,9,44,28,50,15360,500,20,10,0.03,0.03
3,1001,10,53,28,50,15360,500,20,10,0.03,0.03
4,1001,11,36,28,50,15360,500,20,10,0.03,0.03
...,...,...,...,...,...,...,...,...,...,...,...
1754,1496,9,21,49,50,15360,500,20,10,0.03,0.03
1755,1496,10,18,49,50,15360,500,20,10,0.03,0.03
1756,1496,11,13,49,50,15360,500,20,10,0.03,0.03
1757,1496,12,11,49,50,15360,500,20,10,0.03,0.03


In [103]:
df_messages_merge['costo_extra_sms'] = (
    (df_messages_merge['sms_count'] - df_messages_merge['messages_included'])
    .clip(lower=0) * df_messages_merge['usd_per_message']
)

In [104]:
df_messages_final = df_messages_merge[['user_id', 'month', 'sms_count', 'costo_extra_sms']]

In [105]:
df_messages_final[df_messages_final['costo_extra_sms'] > 0]

,user_id,month,sms_count,costo_extra_sms
3,1001,10,53,0.09
21,1007,8,51,0.03
23,1007,10,59,0.27
38,1014,12,64,0.42
39,1015,12,68,0.54
...,...,...,...,...
1732,1489,11,91,1.23
1733,1489,12,86,1.08
1736,1491,6,54,0.12
1737,1491,7,64,0.42


In [106]:
# Merge de los 3
df_final = pd.merge(df_calls_final, df_internet_final, on=['user_id', 'month'], how='outer')
df_final = pd.merge(df_final, df_messages_final, on=['user_id', 'month'], how='outer')


In [107]:
df_final = df_final.fillna(0)

In [108]:
# merge con df_merged
df_final = pd.merge(df_final, df_merged[['user_id', 'usd_monthly_pay', 'age']], on='user_id', how='left')
df_final


,user_id,month,duration,costo_extra_llamadas,mb_used,costo_extra_datos,sms_count,costo_extra_sms,usd_monthly_pay,age
0,1000,12,116.83,0.0,1901.47,0.000000,11.0,0.00,70,45
1,1001,8,171.14,0.0,6919.15,0.000000,30.0,0.00,20,28
2,1001,9,297.69,0.0,13314.82,0.000000,44.0,0.00,20,28
3,1001,10,374.11,0.0,22330.49,68.071191,53.0,0.09,20,28
4,1001,11,374.02,0.0,18504.30,30.706055,36.0,0.00,20,28
...,...,...,...,...,...,...,...,...,...,...
2288,1498,12,324.77,0.0,23137.69,75.954004,0.0,0.00,20,51
2289,1499,9,306.95,0.0,12984.76,0.000000,0.0,0.00,20,37
2290,1499,10,338.25,0.0,19492.43,40.355762,0.0,0.00,20,37
2291,1499,11,287.91,0.0,15580.97,2.157910,0.0,0.00,20,37


In [109]:
# Calcular el costo total
df_final['costo_total'] = df_final['costo_extra_llamadas'] + df_final['costo_extra_datos'] + df_final['costo_extra_sms'] + df_final['usd_monthly_pay']
df_final

,user_id,month,duration,costo_extra_llamadas,mb_used,costo_extra_datos,sms_count,costo_extra_sms,usd_monthly_pay,age,costo_total
0,1000,12,116.83,0.0,1901.47,0.000000,11.0,0.00,70,45,70.000000
1,1001,8,171.14,0.0,6919.15,0.000000,30.0,0.00,20,28,20.000000
2,1001,9,297.69,0.0,13314.82,0.000000,44.0,0.00,20,28,20.000000
3,1001,10,374.11,0.0,22330.49,68.071191,53.0,0.09,20,28,88.161191
4,1001,11,374.02,0.0,18504.30,30.706055,36.0,0.00,20,28,50.706055
...,...,...,...,...,...,...,...,...,...,...,...
2288,1498,12,324.77,0.0,23137.69,75.954004,0.0,0.00,20,51,95.954004
2289,1499,9,306.95,0.0,12984.76,0.000000,0.0,0.00,20,37,20.000000
2290,1499,10,338.25,0.0,19492.43,40.355762,0.0,0.00,20,37,60.355762
2291,1499,11,287.91,0.0,15580.97,2.157910,0.0,0.00,20,37,22.157910


In [110]:
# mira los datos del usuario 1000
df_final[df_final['user_id'] == 1002]



,user_id,month,duration,costo_extra_llamadas,mb_used,costo_extra_datos,sms_count,costo_extra_sms,usd_monthly_pay,age,costo_total
6,1002,10,53.80,0.0,6552.01,0.000000,15.0,0.0,20,36,20.000000
7,1002,11,359.59,0.0,19345.08,38.916797,32.0,0.0,20,36,58.916797
8,1002,12,337.02,0.0,14396.24,0.000000,41.0,0.0,20,36,20.000000


In [111]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2293 entries, 0 to 2292
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   user_id               2293 non-null   int64  
 1   month                 2293 non-null   int32  
 2   duration              2293 non-null   float64
 3   costo_extra_llamadas  2293 non-null   float64
 4   mb_used               2293 non-null   float64
 5   costo_extra_datos     2293 non-null   float64
 6   sms_count             2293 non-null   float64
 7   costo_extra_sms       2293 non-null   float64
 8   usd_monthly_pay       2293 non-null   int64  
 9   age                   2293 non-null   int64  
 10  costo_total           2293 non-null   float64
dtypes: float64(7), int32(1), int64(3)
memory usage: 188.2 KB


In [112]:
df_final.to_csv("../data/ml/final.csv", index=False)